# Example of Ask/Tell interface with DEAP 

In [1]:
import random
import array
import numpy

from deap import base, benchmarks, creator, tools, algorithms
from deap import cma

## Generate/Update (aka ask/tell)

In [2]:
FITCLSNAME = "FIT_TYPE"
INDCLSNAME = "IND_TYPE"
creator.create(FITCLSNAME, base.Fitness, weights=(-1.0,))
creator.create(INDCLSNAME, list, fitness=creator.__dict__[FITCLSNAME])
# HV_THRESHOLD = 116.0        # 120.777 is Optimal value

In [3]:
NDIM = 2

strategy = cma.Strategy(centroid=[0.0]*NDIM, sigma=1.0)

toolbox = base.Toolbox()
toolbox.register("evaluate", benchmarks.sphere)
toolbox.register("generate", strategy.generate, creator.__dict__[INDCLSNAME])
toolbox.register("update", strategy.update)

In [4]:
stats = None
ngen = 2

logbook = tools.Logbook()
logbook.header = ['gen', 'nevals'] + (stats.fields if stats else [])

for gen in range(ngen):
    # Generate a new population
    population = toolbox.generate()
    print(population, '\n')

[[-1.1181817242175354, -0.3960074472585139], [1.7326433935294574, -0.5631455316394197], [0.20190159062020474, 0.4355054992386995], [0.24855196465660626, -0.10440064208347434], [0.22672611073722526, -1.9579110632980519], [0.13632617921183277, -0.5026771400684753]] 

[[1.0287661316921595, 0.30758574496951785], [-0.72747202364791, -0.28926524345969884], [-0.2897024579699778, -0.11735104794430082], [-0.0674636712274339, 1.4105273271783927], [0.18704212712662607, 0.03042276311602353], [-0.2769007613999105, 0.21026411873891315]] 



In [5]:
fitnesses = toolbox.map(toolbox.evaluate, population)
fitnesses

In [6]:
for ind, fit in zip(population, fitnesses):
    print(ind, fit)
    print(type(fit))
    ind.fitness.values = fit

[1.0287661316921595, 0.30758574496951785] (1.152968744225303,)
<class 'tuple'>
[-0.72747202364791, -0.28926524345969884] (0.6128899262641841,)
<class 'tuple'>
[-0.2897024579699778, -0.11735104794430082] (0.09769878260747233,)
<class 'tuple'>
[-0.0674636712274339, 1.4105273271783927] (1.9941386876525036,)
<class 'tuple'>
[0.18704212712662607, 0.03042276311602353] (0.03591030183566663,)
<class 'tuple'>
[-0.2769007613999105, 0.21026411873891315] (0.12088503129290193,)
<class 'tuple'>


In [7]:
toolbox.update(population)

## Differential Evolution

In [8]:
# Problem dimension
NDIM = 10

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", array.array, typecode='d', fitness=creator.FitnessMin)

In [9]:
toolbox = base.Toolbox()
toolbox.register("attr_float", random.uniform, -3, 3)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, NDIM)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("select", tools.selRandom, k=3)
toolbox.register("evaluate", benchmarks.sphere)

In [10]:
# Differential evolution parameters
CR = 0.25
F = 1  
MU = 300
NGEN = 200    

pop = toolbox.population(n=MU);
hof = tools.HallOfFame(1)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", numpy.mean)
stats.register("std", numpy.std)
stats.register("min", numpy.min)
stats.register("max", numpy.max)

logbook = tools.Logbook()
logbook.header = "gen", "evals", "std", "min", "avg", "max"

# Evaluate the individuals
fitnesses = toolbox.map(toolbox.evaluate, pop)
for ind, fit in zip(pop, fitnesses):
    ind.fitness.values = fit

record = stats.compile(pop)
logbook.record(gen=0, evals=len(pop), **record)
print(logbook.stream)

for g in range(1, NGEN):
    for k, agent in enumerate(pop):
        a,b,c = toolbox.select(pop)
        y = toolbox.clone(agent)
        index = random.randrange(NDIM)
        for i, value in enumerate(agent):
            if i == index or random.random() < CR:
                y[i] = a[i] + F*(b[i]-c[i])
        y.fitness.values = toolbox.evaluate(y)
        if y.fitness > agent.fitness:
            pop[k] = y
    hof.update(pop)
    record = stats.compile(pop)
    logbook.record(gen=g, evals=len(pop), **record)
    print(logbook.stream)

print("Best individual is ", hof[0], hof[0].fitness.values[0])


gen	evals	std    	min    	avg    	max   
0  	300  	8.36759	6.53248	29.2301	63.893
1  	300  	8.34231	6.53248	28.3044	63.893
2  	300  	8.36757	3.3417 	27.4456	63.893
3  	300  	8.19147	3.3417 	26.4221	63.893
4  	300  	7.91929	3.3417 	25.5406	52.2714
5  	300  	7.85309	3.3417 	24.5663	44.5414
6  	300  	7.84191	3.3417 	23.5903	44.5414
7  	300  	7.6953 	3.3417 	22.8467	44.5414
8  	300  	7.51544	3.3417 	22.0238	42.4721
9  	300  	7.31856	3.3417 	21.2106	42.4721
10 	300  	7.05602	3.3417 	20.5445	42.4721
11 	300  	6.78236	3.3417 	19.5857	35.64  
12 	300  	6.71492	3.3417 	18.666 	35.0752
13 	300  	6.49194	3.3417 	17.9906	35.0752
14 	300  	6.1274 	3.3417 	17.2125	35.0752
15 	300  	5.91746	3.3417 	16.3136	35.0752
16 	300  	5.83048	3.3417 	15.7701	35.0752
17 	300  	5.57395	3.3417 	15.0399	35.0752
18 	300  	5.3967 	3.3417 	14.5375	35.0752
19 	300  	5.2124 	3.31779	13.9037	35.0752
20 	300  	5.09596	3.31779	13.3339	35.0752
21 	300  	4.72846	3.31779	12.721 	26.7395
22 	300  	4.51798	3.31779	11.9892	24.96

## Problem with use of brian!

In [11]:
import numpy as np
from brian2 import *
from brian2.equations.equations import (DIFFERENTIAL_EQUATION, Equations,
                                        SingleEquation, PARAMETER)
from brian2.input import TimedArray
from brian2 import NeuronGroup, StateMonitor, store, restore, run, defaultclock, second, Quantity
from brian2.stateupdaters.base import StateUpdateMethod

### Input

In [12]:
input_traces = zeros((10,1))*volt
for i in range(1):
    input_traces[1:,i]=i*10*mV

In [13]:
# Create target current traces
output_traces = 10*nS*input_traces

In [14]:
input = input_traces
output = output_traces

In [15]:
params = np.array([
 [ 1.80869973e-08,  2.50218013e-02],
 [ 1.88373085e-08,  9.89559934e-02], 
 [ 1.88373085e-08,  9.89559934e-02], 
])

### Setup The Model for Optimization

In [16]:
input_var = 'v'
output_var = 'I'

parameter_names = {'g', 'E'}
method = ('linear', 'exponential_euler', 'euler')
t_start = 0*second
popsize, _ = np.shape(params)
dt = 0.1 *ms
defaultclock.dt = dt

In [17]:
model = Equations('''
I = g*(v-E) : amp
g : siemens (constant)
E : volt (constant)
''')

In [18]:
state_update_code = StateUpdateMethod.apply_stateupdater(model, {}, method=method)

INFO       No numerical integration method specified, using method 'linear' (took 0.00s). [brian2.stateupdaters.base.method_choice]


### Required Model Operations

In [19]:
Nsteps, Ntraces = input_traces.shape
# N = popsize * len(parameter_names)
N = popsize
duration = Nsteps*dt

In [20]:
model_without_diffeq = Equations([eq for eq in model.ordered
                                      if eq.type != DIFFERENTIAL_EQUATION])
    
# Add a parameter for each differential equation
diffeq_params = Equations([SingleEquation(PARAMETER, varname, model.dimensions[varname])
                           for varname in model.diff_eq_names])

# Our new model:
model = model_without_diffeq + diffeq_params

# Replace input variable by TimedArray
input_traces = TimedArray(input, dt = dt)

In [21]:
input_unit = input.dim
model = model + Equations(input_var + '= input_var(t,i % Ntraces) : '+ "% s" % repr(input_unit))

# Add criterion with TimedArray
output_traces = TimedArray(output, dt = dt)
error_unit = output.dim**2
model = model + Equations('total_error : %s' % repr(error_unit))

In [22]:
neurons = NeuronGroup(Ntraces*N, model, method = method)
neurons.namespace['input_var'] = input_traces
neurons.namespace['output_var'] = output_traces
neurons.namespace['t_start'] = t_start
neurons.namespace['Ntraces'] = Ntraces

#### Record error  
additional differential equation calculating the error

In [23]:
neurons.run_regularly('total_error +=  (' + output_var + '-output_var(t,i % Ntraces))**2 * int(t>=t_start)',
                      when='end')

# Add the code doing the numerical integration
neurons.run_regularly(state_update_code, when='groups')

# store the state of the network
store()

In [24]:
def parameters_dict(params):
    d = dict()
    for name, value in zip(parameter_names, params.T):
        d[name] = value
            
    return d

In [25]:
def calc_error(params):
    print(params)
    popsize, _ = np.shape(params)
    N = popsize

#     neurons = NeuronGroup(Ntraces*N, model, method = method)
    neurons = NeuronGroup(N, model, method = method)
    neurons.namespace['input_var'] = input_traces
    neurons.namespace['output_var'] = output_traces
    neurons.namespace['t_start'] = t_start
    neurons.namespace['Ntraces'] = Ntraces

    # Record error
    neurons.run_regularly('total_error +=  (' + output_var + '-output_var(t,i % Ntraces))**2 * int(t>=t_start)',
                          when='end')

    # Add the code doing the numerical integration
    neurons.run_regularly(state_update_code, when='groups')

    d = parameters_dict(params)
    neurons.set_states(d, units=False)
    run(duration, namespace = {})

    e = neurons.total_error/int((duration-t_start)/defaultclock.dt)
    e = mean(e.reshape((N,Ntraces)),axis=1)
    
    return array(e)

In [26]:
neurons

NeuronGroup(clock=Clock(dt=100. * usecond, name='defaultclock'), when=start, order=0, name='neurongroup')

### Ask and Tel with calc_error Function

In [27]:
start_scope()

In [28]:
model = Equations('''
I = g*(v-E) : amp
g : siemens (constant)
E : volt (constant)
''')

In [29]:
state_update_code = StateUpdateMethod.apply_stateupdater(model, {}, method=method)

In [30]:
model_without_diffeq = Equations([eq for eq in model.ordered
                                      if eq.type != DIFFERENTIAL_EQUATION])
    
# Add a parameter for each differential equation
diffeq_params = Equations([SingleEquation(PARAMETER, varname, model.dimensions[varname])
                           for varname in model.diff_eq_names])

# Our new model:
model = model_without_diffeq + diffeq_params

# Replace input variable by TimedArray
input_traces = TimedArray(input, dt = dt)

In [31]:
input_unit = input.dim
model = model + Equations(input_var + '= input_var(t,i % Ntraces) : '+ "% s" % repr(input_unit))

# Add criterion with TimedArray
output_traces = TimedArray(output, dt = dt)
error_unit = output.dim**2
model = model + Equations('total_error : %s' % repr(error_unit))

### set up DEAP

In [32]:
NDIM = 2
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", array.array, typecode='d', fitness=creator.FitnessMin)

strategy = cma.Strategy(centroid=[0.0]*NDIM, sigma=1.0)

toolbox = base.Toolbox()
toolbox.register("evaluate", benchmarks.sphere)
toolbox.register("generate", strategy.generate, creator.__dict__[INDCLSNAME])
toolbox.register("update", strategy.update)

In [33]:
hof = tools.HallOfFame(1)
stats = tools.Statistics(lambda ind: ind.fitness.values)

In [34]:
stats = None
ngen = 1

logbook = tools.Logbook()
logbook.header = ['gen', 'nevals'] + (stats.fields if stats else [])

for gen in range(ngen):
    # Generate a new population
    population = toolbox.generate()
    print(population, '\n')

[[-0.12303182119136652, -0.35285769435042047], [-1.0874706190900092, 0.8396581486753585], [1.9421244382504428, 0.09595131293367264], [-0.23711706480165134, -0.654145810686827], [-0.4223480943256372, 0.4969505881062848], [1.0049069616305315, -0.3224481136006225]] 



In [35]:
population

[[-0.12303182119136652, -0.35285769435042047],
 [-1.0874706190900092, 0.8396581486753585],
 [1.9421244382504428, 0.09595131293367264],
 [-0.23711706480165134, -0.654145810686827],
 [-0.4223480943256372, 0.4969505881062848],
 [1.0049069616305315, -0.3224481136006225]]

In [36]:
fitnesses = calc_error(np.array(population))

[[-0.12303182 -0.35285769]
 [-1.08747062  0.83965815]
 [ 1.94212444  0.09595131]
 [-0.23711706 -0.65414581]
 [-0.42234809  0.49695059]
 [ 1.00490696 -0.32244811]]


In [37]:
hof.update(population)

In [38]:
for ind, fit in zip(population, fitnesses):
    print(ind, fit)
    ind.fitness.values = (fit,)

[-0.12303182119136652, -0.35285769435042047] 0.0018846646708547197
[-1.0874706190900092, 0.8396581486753585] 0.8337581236370715
[1.9421244382504428, 0.09595131293367264] 0.03472609505300484
[-0.23711706480165134, -0.654145810686827] 0.024058843630887776
[-0.4223480943256372, 0.4969505881062848] 0.04405218918698917
[1.0049069616305315, -0.3224481136006225] 0.10499567039236266


In [39]:
toolbox.update(population)

In [40]:
record = stats.compile(population) if stats is not None else {}
logbook.record(gen=gen, nevals=len(population), **record)

print(logbook.stream)

gen	nevals
0  	6     


In [49]:
hof[0].fitnesses.values[0]

AttributeError: 'IND_TYPE' object has no attribute 'fitnesses'

In [47]:
hof[0].fitness.values[0]

IndexError: tuple index out of range

In [41]:
print("Best individual is ", hof[0], hof[0].fitness.values[0])


IndexError: tuple index out of range